In [1]:
!kaggle datasets download -d sherrytp/airline-delay-analysis -f "airline delay analysis/2018.csv"
!unzip 2018.csv.zip
!rm 2018.csv.zip

 94%|█████████████████████████████████████▊  | 225M/238M [00:05<00:00, 32.0MB/s]
100%|████████████████████████████████████████| 238M/238M [00:05<00:00, 43.3MB/s]
Archive:  2018.csv.zip
  inflating: 2018.csv                


In [3]:
# initlize pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("airline-delay-regression").getOrCreate()

Picked up JAVA_TOOL_OPTIONS:  -Xmx3435m
Picked up JAVA_TOOL_OPTIONS:  -Xmx3435m
22/05/13 18:33:06 WARN Utils: Your hostname, fawazalesay-pysparkairl-ck4s7ekwcta resolves to a loopback address: 127.0.0.1; using 10.0.5.2 instead (on interface ceth0)
22/05/13 18:33:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/13 18:33:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/13 18:33:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/13 18:33:07 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
# Needed to make Jupyter work with Gitpod
import plotly.io as pio
pio.renderers.default = 'iframe_connected'

In [5]:
# Read the data into a dataframe and print the schema
df = spark.read.csv("2018.csv", header=True, inferSchema=True)
df.printSchema()

root
 |-- FL_DATE: string (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_TIME: double (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)
 |-- WHEELS_OFF: double (nullable = true)
 |-- WHEELS_ON: double (nullable = true)
 |-- TAXI_IN: double (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- ARR_TIME: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- CARRIER_DELAY: double (nullable = true)
 |-- WEATHER_DELAY: do

In [9]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

df = df.drop("Unnamed: 27", "LATE_AIRCRAFT_DELAY", "SECURITY_DELAY", "NAS_DELAY", "WEATHER_DELAY", "CARRIER_DELAY", "AIR_TIME", "ACTUAL_ELAPSED_TIME", "DIVERTED", "CANCELLATION_CODE", "CANCELLED", "ARR_TIME", "TAXI_IN", "WHEELS_ON", "WHEELS_OFF", "TAXI_OUT", "DEP_TIME", "OP_CARRIER_FL_NUM")

df.printSchema()

@udf(returnType=StringType())
def get_month(date):
    return date.split("-")[1]

@udf(returnType=StringType())
def get_day(date):
    return date.split("-")[2]

@udf(returnType=StringType())
def get_year(date):
    return date.split("-")[0]

# Adds month and a day column to the dataframe
df = df.withColumn("YEAR", get_year(df["FL_DATE"]))
df = df.withColumn("MONTH", get_month(df["FL_DATE"]))
df = df.withColumn("DAY", get_day(df["FL_DATE"]))

df.drop("FL_DATE")

df.printSchema()

# Print the first 5 rows of the dataframe
df.show(5)

root
 |-- FL_DATE: string (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- MONTH: string (nullable = true)
 |-- DAY: string (nullable = true)

root
 |-- FL_DATE: string (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- MONTH: string (nullable = true)
 |-- DAY: string (nullable = true)

+----------+--